In [2]:
# import statements
import pandas as pd
import numpy as np
import time
from collections import defaultdict
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split,GridSearchCV,cross_validate,cross_val_predict
import sklearn.metrics
from sklearn.metrics import classification_report,confusion_matrix,make_scorer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pickle
from util.client import Nba_Season
# NOTE: scikit-optimize no longer supported, look into using Optuna or Hyperopt
# from skopt import BayesSearchCV
# from skopt.space import Real, Integer, Categorical
from collections import defaultdict
import util.NeuralNet as nn
import autobnn as ab
import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import pyro
import pyro.distributions as dist
from pyro.nn import PyroModule, PyroSample
import torch
import torch.nn as tnn

In [3]:
def load_past_data(files):
    '''
    files: list of tuples in form of (features,samples,is_norm) file paths
    returns: samples,features_norm
    '''
    samples = []
    features_norm = []

    for file in files:
        features_yr = np.genfromtxt(file[0],delimiter=',')
        if file[2]:
            features_norm.extend(features_yr)
        else:
            features_yr_norm = [[float(i)/sum(j) for i in j ]for j in features_yr]
            features_norm.extend(features_yr_norm)
        
        samples_yr = np.genfromtxt(file[1],delimiter=',')
        samples.extend(samples_yr)

    return samples,features_norm

## Introduction
This noteboook will explore the use of traditional neural networks to predict the outcome of NBA games. This notebook will outline the process of generating data using scrapers from ```client.py```, training MLPs on this data, and using these predictions to make bets

## Using client.py
Below are some examples on how to use the utilities in ```client.py```

### 1. Loading from saved data
The NBA folder contains a variety of datasets which can be loaded to train models. ```on_off_stats``` contains pickle data for each season mapping players to their on/off statistics as well as mapping teams to their statistical features. ```samps_feats``` contains samples and features for a given season. With this data, we can populate an Nba_Season object which can be used to add betting data.

In [3]:
with open('../NBA/on_off_stats/2022-2023_on_off.pkl', 'rb') as f:
    loaded_on_off = pickle.load(f)

with open('../NBA/on_off_stats/2022-2023_team_stats.pkl', 'rb') as f:
    loaded_stats = pickle.load(f)

features = np.genfromtxt('../NBA/samps_feats/2022-2023_nba_features_inj.csv',delimiter=',')

samples = np.genfromtxt('../NBA/samps_feats/2022-2023_nba_samples_inj.csv',delimiter=',')

nba_szn_2022_2023 = Nba_Season('2022','2023', team_stats=loaded_stats, team_on_off=loaded_on_off,
                               features=features, samples=samples)

Note that in this case we can just use the existing features and samples for training our models, making the construction of an Nba_Season object unnecessary for training models.

### 2. Generating features from CSV of games in form of [date, away team, away pts, home team, home pts]
In order to generate new features, we can use the Nba_Season object with the paramaters of (start_year, end_year) to generate on/off statistics, features, and samples to our desired output path. To avoid getting flagged and blocked by basketball reference, it is best to run ```pop_const_new()``` and ```generate_features()```in seperate cells with ~1 minute break between.

NOTE: This requires a CSV containing game information from basketball reference in the form of [date, away team, away pts, home team, home pts], for examples on expected format see ```NBA/games```

In [2]:
nba_szn_2023_2024 = Nba_Season('2023','2024')
nba_szn_2023_2024.pop_const_new()

In [3]:
with open('../NBA/on_off_stats/2018-2019_on_off.pkl', 'rb') as f:
    on_off_2018_2019 = pickle.load(f)
with open('../NBA/on_off_stats/2018-2019_team_stats.pkl', 'rb') as f:
    team_stats_2018_2019 = pickle.load(f)

nba_szn_2018_2019 = Nba_Season('2018','2019',team_stats=team_stats_2018_2019,team_on_off=on_off_2018_2019)
features, samples = nba_szn_2018_2019.generate_features('../NBA/games/2018-2019_season_inj.csv')
# save generated data
nba_szn_2018_2019.save_data(save_path='../NBA/samps_feats/')

list index out of range
Failed at line: 44 for file: ../NBA/games/2018-2019_season_inj.csv LINK: https://www.basketball-reference.com/boxscores/201810220DAL.html
list index out of range
Failed at line: 44 for file: ../NBA/games/2018-2019_season_inj.csv LINK: https://www.basketball-reference.com/boxscores/201810220UTA.html
list index out of range
Failed at line: 44 for file: ../NBA/games/2018-2019_season_inj.csv LINK: https://www.basketball-reference.com/boxscores/201810220POR.html
list index out of range
Failed at line: 44 for file: ../NBA/games/2018-2019_season_inj.csv LINK: https://www.basketball-reference.com/boxscores/201810220GSW.html
list index out of range
Failed at line: 44 for file: ../NBA/games/2018-2019_season_inj.csv LINK: https://www.basketball-reference.com/boxscores/201810220LAL.html
list index out of range
Failed at line: 44 for file: ../NBA/games/2018-2019_season_inj.csv LINK: https://www.basketball-reference.com/boxscores/201810230DET.html
Too many failures, terminati

If your IP is blocked the console will output error messages indicating at which entry requests stopped. If this happens, it is best to hide your IP using a VPN and continue generating features using saved pickle data.

In [2]:
with open('../NBA/on_off_stats/2023-2024_on_off.pkl', 'rb') as f:
    on_off_2023_2024 = pickle.load(f)
with open('../NBA/on_off_stats/2023-2024_team_stats.pkl', 'rb') as f:
    team_stats_2023_2024 = pickle.load(f)
    
nba_szn_2023_2024 = Nba_Season('2023','2024',team_stats=team_stats_2023_2024,team_on_off=on_off_2023_2024)
features, samples = nba_szn_2023_2024.generate_features('../NBA/games/2023-2024_nba_season.csv')
# save generated data
nba_szn_2023_2024.save_data(save_path='../NBA/samps_feats/')

### 3. Adding historic bet info from vegas insider
Using our NBA season object can allow us to generate new CSVs containing betting statistics for making bets with trained models

In [ ]:
# https://www.rotowire.com/betting/nba/archive.php - data for over under and spread
games = nba_szn_2023_2024.add_bet_info('../NBA/games/2023-2024_nba_season.csv','../NBA/with_bets/2023-2024_season.csv')

## Testing against historic odds data for 2022-2023
Once a model is trained, we can use the ```test_kelly()``` function defined below to make bets. This function is based on the [Kelly Criterion](https://en.wikipedia.org/wiki/Kelly_criterion) betting strategy.

In [2]:
def kelly(home_pred,away_pred,home_line,away_line,max_bet=100):
    '''
    Applies kelly critereon based on features and moneyline data
    home_pred: Prediction from MLP for home team
    away_pred: Prediction from MLP for away team
    home_line: Moneyline for home team
    away_line: Moneyline for away team
    '''
    bet_amount = 0
    to_win = 0

    log_home = home_pred - home_pred * away_pred / (home_pred + away_pred - (2*home_pred*away_pred))
    log_away = away_pred - home_pred * away_pred / (home_pred + away_pred - (2*home_pred*away_pred))

    # calculate ratio and implied for home
    home_line_adj = home_line
    away_line_adj = away_line
    if home_line < 0:
        home_line_adj *= -1
        home_line_adj /= 100
        home_ratio = 1/(home_line_adj)
        implied_home = home_line_adj/(1+home_line_adj)
    else:
        home_line_adj /= 100
        home_ratio = home_line_adj
        implied_home = 1/(home_line+1)

    # calculate ratio and implied for away
    if away_line < 0:
        away_line_adj *= -1
        away_line_adj /= 100
        away_ratio = 1/(away_line_adj)
        implied_away = away_line_adj/(1+away_line_adj)
    else:
        away_line_adj /= 100
        away_ratio = away_line_adj
        implied_away = 1/(away_line_adj+1)
    
    diff_home = log_home - implied_home
    diff_away = log_away - implied_away

    kelly_home = log_home - (log_away/home_ratio)
    kelly_away = log_away - (log_home/away_ratio)

    prob = 0

    # make bets, negative if away team bet
    if diff_home > diff_away and diff_home > 0.05:
        bet_amount = (max_bet*kelly_home)
        if home_line < 0:
            to_win = bet_amount/((home_line*-1)/100)
        else:
            to_win = bet_amount/((home_line)/100)
        prob = home_pred

    
    elif diff_away > diff_home and diff_away > 0.05:
        bet_amount = (max_bet*kelly_away)
        if away_line < 0:
            to_win = -1*bet_amount/((away_line*-1)/100)
        else:
            to_win = -1*bet_amount/((away_line)/100)
        prob = away_pred

    return bet_amount,to_win,prob

def test_kelly(model,samps,feats,money_lines):
    money_made = 0
    money_risked = 0
    correct = 0
    guessed = 0
    team_bet = []
    amount = []
    gained = []
    probs = []

    for i in range(len(samps)):
        preds = model.predict_proba(feats[i].reshape(1,-1))
        away_pred = preds[0][0]
        home_pred = preds[0][1]
        home_ml = money_lines[i][7]
        away_ml = money_lines[i][10]

        to_bet,to_win,prob = kelly(home_pred,away_pred,home_ml,away_ml)
        probs.append(prob)
        money_risked += to_bet

        curr_gained = 0

        if to_win < 0:
            team_bet.append('Away')
            amount.append(to_bet)
            guessed += 1
            curr_gained = -1*to_bet
            if samps[i] == 1:
                correct += 1
                curr_gained = (-1*to_win)
                #money_made += curr_gained
        elif to_win > 0:
            team_bet.append('Home')
            amount.append(to_bet)
            guessed += 1
            curr_gained = -1*to_bet
            if samps[i] == 0:
                correct += 1
                curr_gained = to_win
                #money_made += curr_gained
        else:
            team_bet.append(0)
            amount.append(0)

        gained.append(curr_gained)

        if curr_gained > 0:
            money_made += curr_gained

    return correct,guessed,team_bet,probs,amount,gained

### Training a model
Below, we train a neural network defined in ```util/NeuralNet.py``` on game data from 2015-2023. First we load the data, then test a variety of hyper parameters to optimize our model.

In [3]:
# load old samples and features
features_norm = np.genfromtxt('../NBA/samps_feats/2015-2023_nba_features_norm_inj.csv',delimiter=',')
samples = np.genfromtxt('../NBA/samps_feats/2015-2023_nba_samples_inj.csv',delimiter=',')
samples_1d = [0 if j[0] == 0 else 1 for j in samples]
feat_train, feat_test, samp_train, samp_test = train_test_split(features_norm,samples_1d, test_size=0.25, random_state=1)

In [8]:
# try my own NN implementation
nba_df = pd.read_csv('../NBA/samps_feats/2015-2023_nba_features_norm_inj.csv',header=None)
#samp = pd.read_csv('samps_feats/2015-2023_nba_samples_inj.csv',header=None)
nba_df['class'] = samples_1d
nba_df.insert(0,'bias',1)

k = 10
nba_class_0 = nba_df.loc[nba_df['class'] == 0].sample(frac=1)
nba_class_0['class'] = [[0,1]] * len(nba_class_0)
nba0_split = np.array_split(nba_class_0,k)
nba_class_1 = nba_df.loc[nba_df['class'] == 1].sample(frac=1)
nba_class_1['class'] = [[1,0]] * len(nba_class_1)
nba1_split = np.array_split(nba_class_1,k)
nba_vals = [[0,1],[1,0]]

nba_fold = []
for i in range(k):
    this_fold = [nba0_split[i],nba1_split[i]]
    nba_fold.append(pd.concat(this_fold))

dig_nn_arc = [[16,64,2],[16,32,2],[16,32,64,2]]

def dig_test(fold,vals,nn_arc,lamb,eps,alpha,batch_size):
    dig_res = nn.k_fold(fold,vals,nn_arc,lamb,eps,alpha,batch_size)
    arc_dict = defaultdict(list)
    print(f'lamb = {lamb} eps = {eps} alpha = {alpha} batch_size = {batch_size}')

    for arc,perf in dig_res.items():
        avg_acc,avg_f1 = [0,0]
        for res in perf:
            avg_acc += res[0]
            avg_f1 += res[1]
        arc_dict['Architecture'].append(arc)
        arc_dict['Accuracy'].append(avg_acc/10)
        arc_dict['F1'].append(avg_f1/10)

    arc_table = pd.DataFrame(arc_dict)
    print(arc_table)

In [10]:
hyper_params = [[0.05,0.001,0.1,200],[0.05,0.001,0.5,200],[0.05,0.0001,1,200]]
for params in hyper_params:
    dig_test(nba_fold,nba_vals,dig_nn_arc,params[0],params[1],params[2],params[3])

lamb = 0.05 eps = 0.001 alpha = 1 batch_size = 200
      Architecture  Accuracy        F1
0      [16, 64, 2]  0.494139  0.440280
1      [16, 32, 2]  0.497437  0.491444
2  [16, 32, 64, 2]  0.488669  0.405804
lamb = 0.05 eps = 0.001 alpha = 2 batch_size = 200
      Architecture  Accuracy        F1
0      [16, 64, 2]  0.489104  0.394730
1      [16, 32, 2]  0.486670  0.426037
2  [16, 32, 64, 2]  0.490318  0.360376
lamb = 0.05 eps = 0.0001 alpha = 3 batch_size = 200
      Architecture  Accuracy        F1
0      [16, 64, 2]  0.489624  0.368952
1      [16, 32, 2]  0.489539  0.388998
2  [16, 32, 64, 2]  0.490406  0.340295


In [12]:
dig_nn_arc = [[16,64,2],[16,32,2],[16,32,16,2]]
hyper_params = [[0.05,0.001,0.1,200],[0.05,0.001,0.5,200],[0.05,0.0001,1,200]]
for params in hyper_params:
    dig_test(nba_fold,nba_vals,dig_nn_arc,params[0],params[1],params[2],params[3])

lamb = 0.05 eps = 0.001 alpha = 0.1 batch_size = 200
      Architecture  Accuracy        F1
0      [16, 64, 2]  0.500391  0.497137
1      [16, 32, 2]  0.495527  0.489780
2  [16, 32, 16, 2]  0.506468  0.502972
lamb = 0.05 eps = 0.001 alpha = 0.5 batch_size = 200
      Architecture  Accuracy        F1
0      [16, 64, 2]  0.492228  0.478710
1      [16, 32, 2]  0.498562  0.495824
2  [16, 32, 16, 2]  0.500999  0.497117
lamb = 0.05 eps = 0.0001 alpha = 1 batch_size = 200
      Architecture  Accuracy        F1
0      [16, 64, 2]  0.491709  0.443859
1      [16, 32, 2]  0.492232  0.456464
2  [16, 32, 16, 2]  0.494660  0.484062


In [13]:
dig_nn_arc = [[16,64,2],[16,32,2],[16,32,16,2]]
hyper_params = [[0.05,0.001,0.1,100],[0.1,0.001,0.1,100],[0.5,0.0001,0.1,100]]
for params in hyper_params:
    dig_test(nba_fold,nba_vals,dig_nn_arc,params[0],params[1],params[2],params[3])

lamb = 0.05 eps = 0.001 alpha = 0.1 batch_size = 100
      Architecture  Accuracy        F1
0      [16, 64, 2]  0.499003  0.496878
1      [16, 32, 2]  0.503514  0.500165
2  [16, 32, 16, 2]  0.510464  0.503028
lamb = 0.1 eps = 0.001 alpha = 0.1 batch_size = 100
      Architecture  Accuracy        F1
0      [16, 64, 2]  0.493011  0.490122
1      [16, 32, 2]  0.502561  0.497471
2  [16, 32, 16, 2]  0.501261  0.493445
lamb = 0.5 eps = 0.0001 alpha = 0.1 batch_size = 100
      Architecture  Accuracy        F1
0      [16, 64, 2]  0.494660  0.493091
1      [16, 32, 2]  0.498302  0.489409
2  [16, 32, 16, 2]  0.501520  0.493224


In [6]:
mlp = MLPClassifier(hidden_layer_sizes=(128,256,256,128), alpha=0.75, learning_rate_init=0.01, activation='identity', tol=0.01,early_stopping=True,epsilon=0.0001,solver='adam',max_iter=500)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TRAINING SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TEST SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

TRAINING SET 

TN, FP, FN, TP
[2678 1722 2519 1718]
              precision    recall  f1-score   support

           0       0.52      0.61      0.56      4400
           1       0.50      0.41      0.45      4237

    accuracy                           0.51      8637
   macro avg       0.51      0.51      0.50      8637
weighted avg       0.51      0.51      0.50      8637

TEST SET 

TN, FP, FN, TP
[880 611 835 554]
              precision    recall  f1-score   support

           0       0.51      0.59      0.55      1491
           1       0.48      0.40      0.43      1389

    accuracy                           0.50      2880
   macro avg       0.49      0.49      0.49      2880
weighted avg       0.49      0.50      0.49      2880



Once we are satisfied with our model's performance, we can make our bets using betting data

In [11]:
# load features, for each feature predict outcome, use prediction for kelly bet on ML
feats = np.genfromtxt('../NBA/samps_feats/2022-2023_nba_features_inj.csv',delimiter=',')
samps = np.genfromtxt('../NBA/samps_feats/2022-2023_nba_samples_inj.csv',delimiter=',')
bet_data = np.genfromtxt('../NBA/with_bets/2022-2023_season.csv',delimiter=',')
samps_1d = [0 if j[0] == 0 else 1 for j in samps]
correct,guessed,team_bet,probs,amount,gained = test_kelly(mlp,samps_1d,feats,bet_data[1:])

print(correct)
print(guessed)
print(sum(gained))

602
1285
-36628.49345755003


In [29]:
preds = mlp.predict(feats)
print(confusion_matrix(samps_1d,preds).ravel())
print(classification_report(samps_1d,preds))

[405 314 365 236]
              precision    recall  f1-score   support

           0       0.53      0.56      0.54       719
           1       0.43      0.39      0.41       601

    accuracy                           0.49      1320
   macro avg       0.48      0.48      0.48      1320
weighted avg       0.48      0.49      0.48      1320



As we can see, our model is quite reckless in gambling, likely due to the fact that it is extremely overconfident in its predictions.

## Generate samples and features for current season

In [30]:
curr_szn = Nba_Season('2023','2024')
curr_szn.pop_const_new()
feats_curr, samps_curr = curr_szn.generate_features('../NBA/games/2023-2024_nba_season.csv')
# save generated data
curr_szn.save_data(save_path='../NBA/samps_feats/')
curr_bet_data = curr_szn.add_bet_info('../NBA/2023-2024_nba_season.csv','../NBA/with_bets/2023-2024_season.csv')
samps_1d_curr = [0 if j[0] == 0 else 1 for j in samps_curr]

In [67]:
preds = mlp.predict(feats_curr)
print(confusion_matrix(samps_1d_curr,preds).ravel())
print(classification_report(samps_1d_curr,preds))

[248 161 208  99]
              precision    recall  f1-score   support

           0       0.54      0.61      0.57       409
           1       0.38      0.32      0.35       307

    accuracy                           0.48       716
   macro avg       0.46      0.46      0.46       716
weighted avg       0.47      0.48      0.48       716



In [66]:
bet_data = np.genfromtxt('../NBA/with_bets/2023-2024_season.csv',delimiter=',')
fix_bet_data = bet_data[1:]
samps_1d_curr = [1 if j[2] > j[4] else 0 for j in fix_bet_data]
correct,guessed,team_bet,probs,amount,gained = test_kelly(mlp,samps_1d_curr,feats_curr,fix_bet_data)

df = pd.DataFrame(curr_bet_data)
df['team_bet'] = team_bet
df['confidence'] = probs
df['amount'] = amount
df['gained'] = gained

print(df['gained'].sum())

-13786.679351604225


## 2013 - 2023 NBA Seasons

In [ ]:
# TODO fix scraper for this season!
nba_szn_2014 = Nba_Season('2013','2014')
nba_szn_2014.pop_const_new()

In [ ]:
features_2014, samples_2014 = nba_szn_2014.generate_features('../NBA/games/2013-2014_season_inj.csv')
nba_szn_2014.save_data(save_path='../NBA/samps_feats/')

In [2]:
# load old samples and features
features_norm = np.genfromtxt('../NBA/samps_feats/2015-2023_nba_features_norm_inj.csv',delimiter=',')
samples = np.genfromtxt('../NBA/samps_feats/2015-2023_nba_samples_inj.csv',delimiter=',')

## 2014 - 2023 NBA Seasons

In [3]:
samples_1d = [0 if j[0] == 0 else 1 for j in samples]
feat_train, feat_test, samp_train, samp_test = train_test_split(features_norm,samples_1d, test_size=0.25, random_state=1)

### Searching for hyperparams
In an attempt to find the optimal hyperparameters for our model, we can run BayesSearch

In [14]:
opt = BayesSearchCV(
    MLPClassifier(hidden_layer_sizes=(128,256,256,128)),
    {
        'solver': ['adam'],
        'learning_rate_init': (0.00001,0.1),
        'learning_rate' :['invscaling','constant'],
        'max_iter': (500,5000),
        'activation': ['logistic', 'tanh', 'relu'],
        'alpha': (1e-6,2),
        'tol' : (1e-6,0.01)
    },
    n_iter=32,
    cv=5
)

opt.fit(feat_train,samp_train)
print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(feat_test, samp_test))

val. score: 0.5094361877801368
test score: 0.5177083333333333


In [16]:
print(opt.best_params_)

OrderedDict([('activation', 'relu'), ('alpha', 5.830439210684934), ('learning_rate', 'invscaling'), ('learning_rate_init', 0.06113148894211121), ('max_iter', 4524), ('solver', 'adam'), ('tol', 0.006365456815293766)])


In [42]:
mlp = MLPClassifier(hidden_layer_sizes=(128,256,256,128), alpha=0.00075, learning_rate_init=0.0006113148894211121, activation='tanh', tol=0.00006365456815293766, solver='lbfgs',max_iter=4524)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TRAINING SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TEST SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


TRAINING SET 

TN, FP, FN, TP
[4393    7    8 4229]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4400
           1       1.00      1.00      1.00      4237

    accuracy                           1.00      8637
   macro avg       1.00      1.00      1.00      8637
weighted avg       1.00      1.00      1.00      8637

TEST SET 

TN, FP, FN, TP
[768 723 694 695]
              precision    recall  f1-score   support

           0       0.53      0.52      0.52      1491
           1       0.49      0.50      0.50      1389

    accuracy                           0.51      2880
   macro avg       0.51      0.51      0.51      2880
weighted avg       0.51      0.51      0.51      2880



In [5]:
mlp = MLPClassifier(hidden_layer_sizes=(128,256,256,128), alpha=0.0075, learning_rate_init=0.0006113148894211121, activation='tanh', tol=0.00006365456815293766, solver='lbfgs',max_iter=4524,early_stopping=True)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TRAINING SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TEST SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


TRAINING SET 

TN, FP, FN, TP
[4389   11   15 4222]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4400
           1       1.00      1.00      1.00      4237

    accuracy                           1.00      8637
   macro avg       1.00      1.00      1.00      8637
weighted avg       1.00      1.00      1.00      8637

TEST SET 

TN, FP, FN, TP
[720 771 713 676]
              precision    recall  f1-score   support

           0       0.50      0.48      0.49      1491
           1       0.47      0.49      0.48      1389

    accuracy                           0.48      2880
   macro avg       0.48      0.48      0.48      2880
weighted avg       0.49      0.48      0.48      2880



In [51]:
mlp = MLPClassifier(hidden_layer_sizes=(128,256,256,128), alpha=5.830439210684934, learning_rate_init=0.0006113148894211121, activation='tanh', tol=0.00006365456815293766, solver='lbfgs',max_iter=4524)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TRAINING SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TEST SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


TRAINING SET 

TN, FP, FN, TP
[3272 1128 2230 2007]
              precision    recall  f1-score   support

           0       0.59      0.74      0.66      4400
           1       0.64      0.47      0.54      4237

    accuracy                           0.61      8637
   macro avg       0.62      0.61      0.60      8637
weighted avg       0.62      0.61      0.60      8637

TEST SET 

TN, FP, FN, TP
[942 549 907 482]
              precision    recall  f1-score   support

           0       0.51      0.63      0.56      1491
           1       0.47      0.35      0.40      1389

    accuracy                           0.49      2880
   macro avg       0.49      0.49      0.48      2880
weighted avg       0.49      0.49      0.48      2880



In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(16,32,64,64,32,16), alpha=0.0001, learning_rate='invscaling', activation='tanh', solver='adam',max_iter=10000)
cv_res = cross_validate(mlp,features_norm,samples_1d,cv=10,scoring=mlp.score)
print("Fit scores: {}".format(cv_res['test_score']))

In [9]:
mlp = MLPClassifier(hidden_layer_sizes=(16,32,64,64,32,16), alpha=0.0001, learning_rate='invscaling', activation='tanh', solver='adam',max_iter=10000)
pred = cross_val_predict(mlp,features_norm,samples_1d,cv=10)
print('TN, FP, FN, TP')
print(confusion_matrix(samples_1d,pred).ravel())
print(classification_report(samples_1d,pred))

TN, FP, FN, TP
[3099 2792 3062 2564]
              precision    recall  f1-score   support

           0       0.50      0.53      0.51      5891
           1       0.48      0.46      0.47      5626

    accuracy                           0.49     11517
   macro avg       0.49      0.49      0.49     11517
weighted avg       0.49      0.49      0.49     11517



In [11]:
GRID = [
    {'scaler': [StandardScaler()],
     'estimator': [MLPClassifier(random_state=1)],
     'estimator__solver': ['adam','lbfgs'],
     'estimator__learning_rate_init': [0.0001,0.001,0.01],
     'estimator__learning_rate' :['invscaling','constant'],
     'estimator__max_iter': [500,1500,2500],
     'estimator__hidden_layer_sizes': [(128,128,128),(128,256,128),(64,128,128,64),(16,32,64,64,32,16)],
     'estimator__activation': ['logistic', 'tanh', 'relu'],
     'estimator__alpha': [0.0001, 0.001, 0.01,0.1],
     'estimator__tol' : [0.01, 0.0001, 0.001],
     'estimator__early_stopping': [True, False]
     }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', MLPClassifier())])

grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, 
                            scoring=make_scorer(sklearn.metrics.accuracy_score),# average='macro'), 
                            n_jobs=-1, refit=True, verbose=1, 
                            return_train_score=False)

grid_search.fit(feat_train,samp_train)
print('Best parameters found:\n', grid_search.best_params_)
print('Best parameters score:\n', grid_search.best_score_)

Fitting 5 folds for each of 10368 candidates, totalling 51840 fits


In [32]:
mlp = MLPClassifier(hidden_layer_sizes=(16,32,64,64,32,16), alpha=0.0075, learning_rate='invscaling', activation='tanh', tol=0.001, solver='lbfgs',max_iter=1500)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TRAINING SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TEST SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

TRAINING SET 

TN, FP, FN, TP
[3451  949 3242  995]
              precision    recall  f1-score   support

           0       0.52      0.78      0.62      4400
           1       0.51      0.23      0.32      4237

    accuracy                           0.51      8637
   macro avg       0.51      0.51      0.47      8637
weighted avg       0.51      0.51      0.47      8637

TEST SET 

TN, FP, FN, TP
[1157  334 1097  292]
              precision    recall  f1-score   support

           0       0.51      0.78      0.62      1491
           1       0.47      0.21      0.29      1389

    accuracy                           0.50      2880
   macro avg       0.49      0.49      0.45      2880
weighted avg       0.49      0.50      0.46      2880



In [33]:
mlp = MLPClassifier(hidden_layer_sizes=(16,32,64,64,32,16), alpha=0.0001, learning_rate='invscaling', activation='tanh', solver='adam',max_iter=10000)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TRAINING SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TEST SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

TRAINING SET 

TN, FP, FN, TP
[3768  632  672 3565]
              precision    recall  f1-score   support

           0       0.85      0.86      0.85      4400
           1       0.85      0.84      0.85      4237

    accuracy                           0.85      8637
   macro avg       0.85      0.85      0.85      8637
weighted avg       0.85      0.85      0.85      8637

TEST SET 

TN, FP, FN, TP
[753 738 713 676]
              precision    recall  f1-score   support

           0       0.51      0.51      0.51      1491
           1       0.48      0.49      0.48      1389

    accuracy                           0.50      2880
   macro avg       0.50      0.50      0.50      2880
weighted avg       0.50      0.50      0.50      2880



In [19]:
mlp = MLPClassifier(hidden_layer_sizes=(32,64,128,64,32), alpha=0.5, learning_rate='invscaling', activation='relu', tol=0.001,solver='lbfgs',max_iter=2500)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TRAINING SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TEST SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

TRAINING SET 

TN, FP, FN, TP
[3609  496  504 3452]
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      4105
           1       0.87      0.87      0.87      3956

    accuracy                           0.88      8061
   macro avg       0.88      0.88      0.88      8061
weighted avg       0.88      0.88      0.88      8061

TEST SET 

TN, FP, FN, TP
[920 866 887 783]
              precision    recall  f1-score   support

           0       0.51      0.52      0.51      1786
           1       0.47      0.47      0.47      1670

    accuracy                           0.49      3456
   macro avg       0.49      0.49      0.49      3456
weighted avg       0.49      0.49      0.49      3456



C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [20]:
GRID = [
    {'scaler': [StandardScaler()],
     'estimator': [MLPClassifier(random_state=1)],
     'estimator__solver': ['adam'],
     'estimator__learning_rate_init': [0.0001,0.001,0.01],
     'estimator__learning_rate' :['invscaling','constant'],
     'estimator__max_iter': [10000],
     'estimator__hidden_layer_sizes': [(32,64,32), (64,64,64), (64,128,64), (32,64,64,32), (16,32,64,32,16), (32,64,128,64,32), (16,32,64,64,32,16)],
     'estimator__activation': ['logistic', 'tanh', 'relu'],
     'estimator__alpha': [0.0001, 0.001, 0.005],
     'estimator__tol' : [0.01, 0.0001, 0.001],
     'estimator__early_stopping': [True, False]
     }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', MLPClassifier())])

grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, 
                            scoring=make_scorer(sklearn.metrics.accuracy_score),# average='macro'), 
                            n_jobs=-1, refit=True, verbose=1, 
                            return_train_score=False)

grid_search.fit(feat_train,samp_train)
print('Best parameters found:\n', grid_search.best_params_)
print('Best parameters score:\n', grid_search.best_score_)

Fitting 5 folds for each of 2268 candidates, totalling 11340 fits
Best parameters found:
 {'estimator': MLPClassifier(alpha=0.005, hidden_layer_sizes=(16, 32, 64, 32, 16),
              learning_rate='invscaling', max_iter=10000, random_state=1), 'estimator__activation': 'relu', 'estimator__alpha': 0.005, 'estimator__early_stopping': False, 'estimator__hidden_layer_sizes': (16, 32, 64, 32, 16), 'estimator__learning_rate': 'invscaling', 'estimator__learning_rate_init': 0.001, 'estimator__max_iter': 10000, 'estimator__solver': 'adam', 'estimator__tol': 0.0001, 'scaler': StandardScaler()}
Best parameters score:
 0.5117242965422075


In [27]:
mlp = MLPClassifier(hidden_layer_sizes=(128,256,256,128), alpha=0.005, learning_rate='invscaling', learning_rate_init=0.001, activation='relu', tol=0.0001,solver='adam',max_iter=10000)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TRAINING SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TEST SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

TRAINING SET 

TN, FP, FN, TP
[3902  498  431 3806]
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      4400
           1       0.88      0.90      0.89      4237

    accuracy                           0.89      8637
   macro avg       0.89      0.89      0.89      8637
weighted avg       0.89      0.89      0.89      8637

TEST SET 

TN, FP, FN, TP
[743 748 672 717]
              precision    recall  f1-score   support

           0       0.53      0.50      0.51      1491
           1       0.49      0.52      0.50      1389

    accuracy                           0.51      2880
   macro avg       0.51      0.51      0.51      2880
weighted avg       0.51      0.51      0.51      2880



In [41]:
mlp = MLPClassifier(hidden_layer_sizes=(128,256,512,256,128), alpha=0.005, learning_rate='invscaling', learning_rate_init=0.00001, activation='relu',solver='adam',max_iter=10000)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TRAINING SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TEST SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

TRAINING SET 

TN, FP, FN, TP
[3370 1030 1456 2781]
              precision    recall  f1-score   support

           0       0.70      0.77      0.73      4400
           1       0.73      0.66      0.69      4237

    accuracy                           0.71      8637
   macro avg       0.71      0.71      0.71      8637
weighted avg       0.71      0.71      0.71      8637

TEST SET 

TN, FP, FN, TP
[824 667 763 626]
              precision    recall  f1-score   support

           0       0.52      0.55      0.54      1491
           1       0.48      0.45      0.47      1389

    accuracy                           0.50      2880
   macro avg       0.50      0.50      0.50      2880
weighted avg       0.50      0.50      0.50      2880



## 2022-2023 NBA Season with Injuries 

In [23]:
features_norm = []
samples = []

pop_team_stats('2023')

time.sleep(15)

pop_team_on_off('2023')

time.sleep(15)

# with open('2022-2023_on_off.pkl', 'wb') as f:
#     pickle.dump(TEAM_ON_OFF, f)

# with open('2022-2023_on_off.pkl', 'rb') as f:
#     loaded_dict = pickle.load(f)

features_2023,samples_2023 = generate_features('2023','../NBA/games/2022-2023_season_injury.csv')

np.savetxt('../NBA/2022-2023_nba_features_inj.csv', features_2023, delimiter=',')
np.savetxt('../NBA/2022-2023_nba_samples_inj.csv', samples_2023, delimiter=',')

#features_2023_ext,samples_2023_ext = generate_features('2023','games/2022-2023_season_inj_ext.csv')

#features_2023.extend(features_2023_ext)
#samples_2023.extend(samples_2023_ext)
features_2023_norm = [[float(i)/sum(j) for i in j ]for j in features_2023]

features_norm.extend(features_2023_norm)
samples.extend(samples_2023)

#features_2023_norm = [[float(i)/sum(j) for i in j ]for j in features_2023]
samples_2023_1d = [0 if j[0] == 0 else 1 for j in samples_2023]
feat_train, feat_test, samp_train, samp_test = train_test_split(features_2023_norm,samples_2023_1d, test_size=0.30, random_state=1)

In [58]:
mlp = MLPClassifier(hidden_layer_sizes=(16,32,64,64,32,16), alpha=0.075, learning_rate='invscaling', activation='tanh', solver='lbfgs',max_iter=10000)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

TN, FP, FN, TP
[496   1   1 426]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       497
           1       1.00      1.00      1.00       427

    accuracy                           1.00       924
   macro avg       1.00      1.00      1.00       924
weighted avg       1.00      1.00      1.00       924

TN, FP, FN, TP
[114 108  85  89]
              precision    recall  f1-score   support

           0       0.57      0.51      0.54       222
           1       0.45      0.51      0.48       174

    accuracy                           0.51       396
   macro avg       0.51      0.51      0.51       396
weighted avg       0.52      0.51      0.51       396



C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [64]:
mlp = MLPClassifier(hidden_layer_sizes=(16,32,64,64,32,16), alpha=0.0001, learning_rate='invscaling', activation='tanh', solver='adam',max_iter=10000)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

TN, FP, FN, TP
[451  46  14 413]
              precision    recall  f1-score   support

           0       0.97      0.91      0.94       497
           1       0.90      0.97      0.93       427

    accuracy                           0.94       924
   macro avg       0.93      0.94      0.93       924
weighted avg       0.94      0.94      0.94       924

TN, FP, FN, TP
[111 111  95  79]
              precision    recall  f1-score   support

           0       0.54      0.50      0.52       222
           1       0.42      0.45      0.43       174

    accuracy                           0.48       396
   macro avg       0.48      0.48      0.48       396
weighted avg       0.48      0.48      0.48       396



In [65]:
arcs = [(128,256,128),(64,128,128,64),(128,256,256,128)]
for arc in arcs:
    mlp = MLPClassifier(hidden_layer_sizes=arc, alpha=0.0075, learning_rate_init=0.001, activation='tanh', solver='adam', epsilon=0.0000001, max_iter=10000)
    mlp.fit(feat_train,samp_train)
    print(arc)
    predict_train = mlp.predict(feat_train)
    predict_test = mlp.predict(feat_test)

    print('TN, FP, FN, TP')
    print(confusion_matrix(samp_train,predict_train).ravel())
    print(classification_report(samp_train,predict_train))
    print('TN, FP, FN, TP')
    print(confusion_matrix(samp_test,predict_test).ravel())
    print(classification_report(samp_test,predict_test))


(128, 256, 128)
TN, FP, FN, TP
[487  10   8 419]
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       497
           1       0.98      0.98      0.98       427

    accuracy                           0.98       924
   macro avg       0.98      0.98      0.98       924
weighted avg       0.98      0.98      0.98       924

TN, FP, FN, TP
[118 104  94  80]
              precision    recall  f1-score   support

           0       0.56      0.53      0.54       222
           1       0.43      0.46      0.45       174

    accuracy                           0.50       396
   macro avg       0.50      0.50      0.50       396
weighted avg       0.50      0.50      0.50       396

(64, 128, 128, 64)
TN, FP, FN, TP
[488   9  29 398]
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       497
           1       0.98      0.93      0.95       427

    accuracy                           0.96     

In [68]:
mlp = MLPClassifier(hidden_layer_sizes=(16,32,64,64,32,16), alpha=0.05, learning_rate='invscaling', activation='tanh', solver='lbfgs',max_iter=5000)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print(classification_report(samp_train,predict_train))
print(classification_report(samp_test,predict_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       497
           1       1.00      1.00      1.00       427

    accuracy                           1.00       924
   macro avg       1.00      1.00      1.00       924
weighted avg       1.00      1.00      1.00       924

              precision    recall  f1-score   support

           0       0.54      0.55      0.55       222
           1       0.41      0.40      0.40       174

    accuracy                           0.48       396
   macro avg       0.48      0.48      0.48       396
weighted avg       0.48      0.48      0.48       396



C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


## 2018 - 2023 NBA Seasons with Injuries

### 1. Generate Samples and Features for each Season

### 2. Test Model and Tune Hyper Paramaters

In [21]:
samples_1d =  [0 if j[0] == 0 else 1 for j in samples]
feat_train, feat_test, samp_train, samp_test = train_test_split(features_norm,samples_1d, test_size=0.30, random_state=1)

In [31]:
mlp = MLPClassifier(hidden_layer_sizes=(16,32,64,64,32,16), alpha=0.0001, learning_rate_init=0.025, learning_rate='adaptive', activation='tanh', solver='adam', epsilon=0.0001, max_iter=10000)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

TN, FP, FN, TP
[2715   76   67 2448]
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      2791
           1       0.97      0.97      0.97      2515

    accuracy                           0.97      5306
   macro avg       0.97      0.97      0.97      5306
weighted avg       0.97      0.97      0.97      5306

TN, FP, FN, TP
[571 597 588 519]
              precision    recall  f1-score   support

           0       0.49      0.49      0.49      1168
           1       0.47      0.47      0.47      1107

    accuracy                           0.48      2275
   macro avg       0.48      0.48      0.48      2275
weighted avg       0.48      0.48      0.48      2275



In [36]:
mlp = MLPClassifier(hidden_layer_sizes=(16,32,64,64,32,16), alpha=0.0001, learning_rate_init=0.0025, learning_rate='adaptive', activation='relu', solver='adam', epsilon=0.000001, max_iter=5000)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

TN, FP, FN, TP
[2441  350 1301 1214]
              precision    recall  f1-score   support

           0       0.65      0.87      0.75      2791
           1       0.78      0.48      0.60      2515

    accuracy                           0.69      5306
   macro avg       0.71      0.68      0.67      5306
weighted avg       0.71      0.69      0.68      5306

TN, FP, FN, TP
[824 344 788 319]
              precision    recall  f1-score   support

           0       0.51      0.71      0.59      1168
           1       0.48      0.29      0.36      1107

    accuracy                           0.50      2275
   macro avg       0.50      0.50      0.48      2275
weighted avg       0.50      0.50      0.48      2275



In [43]:
mlp = MLPClassifier(max_iter=10000)

parameter_space = {
    'hidden_layer_sizes': [(16,32,16), (32,64,32), (64,64,64), (64,128,48), (16,32,64,32,16)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05, 0.001],
    'learning_rate': ['constant','adaptive'],
}

clf = GridSearchCV(mlp,parameter_space,n_jobs=-1)
clf.fit(feat_train,samp_train)

print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'activation': 'relu', 'alpha': 0.05, 'hidden_layer_sizes': (64, 128, 48), 'learning_rate': 'adaptive', 'solver': 'sgd'}


In [55]:
GRID = [
    {'scaler': [StandardScaler()],
     'estimator': [MLPClassifier(random_state=1)],
     'estimator__solver': ['adam'],
     'estimator__learning_rate_init': [0.0001],
     'estimator__max_iter': [10000],
     'estimator__hidden_layer_sizes': [(16,32,16), (32,64,32), (64,64,64), (64,128,48), (16,32,64,32,16)],
     'estimator__activation': ['logistic', 'tanh', 'relu'],
     'estimator__alpha': [0.0001, 0.001, 0.005],
     'estimator__epsilon' : [0.001,0.00001,0.00000001],
     'estimator__tol' : [0.01, 0.0001, 0.000001],
     'estimator__early_stopping': [True, False]
     }
]

PIPELINE = Pipeline([('scaler', None), ('estimator', MLPClassifier())])

grid_search = GridSearchCV(estimator=PIPELINE, param_grid=GRID, 
                            scoring=make_scorer(sklearn.metrics.accuracy_score),# average='macro'), 
                            n_jobs=-1, refit=True, verbose=1, 
                            return_train_score=False)

grid_search.fit(feat_train,samp_train)
print('Best parameters found:\n', grid_search.best_params_)

Fitting 5 folds for each of 810 candidates, totalling 4050 fits
Best parameters found:
 {'estimator': MLPClassifier(hidden_layer_sizes=(32, 64, 32), learning_rate_init=0.0001,
              max_iter=10000, random_state=1), 'estimator__activation': 'relu', 'estimator__alpha': 0.0001, 'estimator__early_stopping': False, 'estimator__epsilon': 1e-08, 'estimator__hidden_layer_sizes': (32, 64, 32), 'estimator__learning_rate_init': 0.0001, 'estimator__max_iter': 10000, 'estimator__solver': 'adam', 'estimator__tol': 0.0001, 'scaler': StandardScaler()}


In [ ]:
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [78]:
mlp = MLPClassifier(hidden_layer_sizes=(32,64,128,64,32), alpha=0.075, learning_rate='invscaling', activation='tanh', solver='lbfgs',max_iter=10000)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

TN, FP, FN, TP
[2787    4   11 2504]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2791
           1       1.00      1.00      1.00      2515

    accuracy                           1.00      5306
   macro avg       1.00      1.00      1.00      5306
weighted avg       1.00      1.00      1.00      5306

TN, FP, FN, TP
[593 575 602 505]
              precision    recall  f1-score   support

           0       0.50      0.51      0.50      1168
           1       0.47      0.46      0.46      1107

    accuracy                           0.48      2275
   macro avg       0.48      0.48      0.48      2275
weighted avg       0.48      0.48      0.48      2275



C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [93]:
mlp = MLPClassifier(hidden_layer_sizes=(32,64,128,64,32), alpha=0.5, learning_rate='invscaling', activation='relu', tol=0.001,solver='lbfgs',max_iter=2500)
mlp.fit(feat_train,samp_train)

predict_train = mlp.predict(feat_train)
predict_test = mlp.predict(feat_test)

print('TRAINING SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_train,predict_train).ravel())
print(classification_report(samp_train,predict_train))
print('TEST SET \n')
print('TN, FP, FN, TP')
print(confusion_matrix(samp_test,predict_test).ravel())
print(classification_report(samp_test,predict_test))

TRAINING SET 

TN, FP, FN, TP
[2637  154  182 2333]
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      2791
           1       0.94      0.93      0.93      2515

    accuracy                           0.94      5306
   macro avg       0.94      0.94      0.94      5306
weighted avg       0.94      0.94      0.94      5306

TEST SET 

TN, FP, FN, TP
[609 559 558 549]
              precision    recall  f1-score   support

           0       0.52      0.52      0.52      1168
           1       0.50      0.50      0.50      1107

    accuracy                           0.51      2275
   macro avg       0.51      0.51      0.51      2275
weighted avg       0.51      0.51      0.51      2275



C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:541: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
